# 2: Trip times from hubs to city center

###### Notebook to calculate trip times & distances from center of the given area towards all other nodes in the given area. The current script calculates trip times for walking. Edit travel_speed & network_type to adjust it for different kinds and ways of transportation.

### Setting up packages and variables

In [1]:
import networkx as nx
import osmnx as ox
ox.config(log_console=True, use_cache=True)
ox.__version__
import pandas as pd
ox.config(use_cache=True, log_console=True)

# configure the place, network type
place = 'Binnenstad, Delft, Netherlands'
network_type = 'all'

# theme colors
miev_pink = '#f06675'
miev_blue = '#2c3174'
miev_yellow = '#fbe56d'

/root/venv/lib/python3.9/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


### Download network

In [2]:
# download the street network with inner city + 1km buffer
G = ox.graph_from_place(place, network_type, buffer_dist=1000)

# download street network for inner city
G_nonbuffer = ox.graph_from_place(place, network_type)

### Find city center and filter nodes outside of city center

In [3]:
# find the centermost node 
gdf_nodes = ox.graph_to_gdfs(G_nonbuffer, edges=False)

# get x and y coordinates of centermost node of the geometry
x = gdf_nodes['geometry'].unary_union.centroid.x
y = gdf_nodes['geometry'].unary_union.centroid.y

# define center node by inputting coordinates (x=lat, y=lng)
center_node = ox.nearest_nodes(G, x, y)

# extract nodes and edges from both graphs (with buffer and without)
nodes1, edges1 = ox.graph_to_gdfs(G)
nodes2, edges2 = ox.graph_to_gdfs(G_nonbuffer)

# Extracts inner city nodes from the 1 km extra buffer city center node list
outer_join = nodes1.merge(nodes2, how = 'outer', indicator = True)
anti_join = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
nodes_new = anti_join

# join the new nodes with existing edges
G_outer = ox.graph_from_gdfs(nodes_new, edges1)

### Shortest path from city center to all nodes in area (takes some time)

In [4]:
# add lengths in meters to all edges 
G = ox.distance.add_edge_lengths(G, precision=3, edges=None)

# make list out of G
G_list = list(G)

# get index of center node
print('Center node index is', G_list.index(center_node) if center_node in G_list else None)
center_node_index =  G_list.index(center_node)

# this almost took all my will to live, in 2 occasions
# empty dataframe necessary to append loop results
df = pd.DataFrame()  

# start index for loop
loop_index = 0

# find max index for when the loop needs to stop
max_index = G_list.index(list(G)[-1])

# loop to find total lengths of shortest path
while loop_index < max_index:
    try: 
        # origin is start location, destination is all nodes that you want a shortest path towards
        orig, dest = list(G)[center_node_index], list(G_list)[loop_index]
        # shortest path function to obtain lenngths
        travel = nx.shortest_path_length(G, orig, dest, weight='length')
        # append results in empty dataframe
        df = df.append({'Index':[loop_index],'Node ID':[dest],
                                'Travel Dis':[travel]},ignore_index=True)

    # exception to print when a node doesn't connect (shouldn't have any of these in principle)   
    except:
        print('no path found')

    # +1 the loop to shortest path towards all nodes
    loop_index += 1

# export to excel
df.to_excel("data/Output.xlsx")

/tmp/ipykernel_331/3593709162.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Index':[loop_index],'Node ID':[dest],
/tmp/ipykernel_331/3593709162.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Index':[loop_index],'Node ID':[dest],
/tmp/ipykernel_331/3593709162.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Index':[loop_index],'Node ID':[dest],
/tmp/ipykernel_331/3593709162.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Index':[loop_index],'Node ID':[dest],
/tmp/ipykernel_331/3593709162.py:29: FutureWarning: The frame.append method is deprecated an

### Setting up dataframe with results and calculating time through distances

In [5]:
df = pd.read_excel('data/Output.xlsx')
df['Travel Dis'] = df['Travel Dis'].str.strip('[]')

# configure travel speed and convert to meter per seconds
travel_speed = int(5.0) #walking speed in km/hour
travel_speed_ms = travel_speed / 3.6

print('travel speed used in calculation: ',travel_speed_ms, 'm/s')

# from object to numeric values to calculate with it (or float)
df['Travel Dis'] = pd.to_numeric(df['Travel Dis'])
# df["Travel Dis"] = df["Travel Dis"].astype(float)

# get travel time in seconds by multiplying travel speed with distance
df["Travel Time"] = df["Travel Dis"] / travel_speed_ms

# convert travel time to minutes (optional)
df["Travel Time"] = df["Travel Time"] / 60.0
df["Travel Time"]


df = df.drop(columns='Unnamed: 0')
df = df.round(decimals=2)

df.to_excel("data/final_distances_citycenter_preextraction.xlsx")

travel speed used in calculation:  1.3888888888888888 m/s


### Extract inner city nodes from shortest path graph (Only needed when you need to extract nodes)

In [6]:
# Shortest path goes over all nodes in the map, extract unwanted nodes (by providing a list of wished nodes) with this script
# for this particular case the nodes within the city center bounds weren't needed, thus these were extracted

df = pd.read_excel("data/final_distances_citycenter_preextraction.xlsx")

Node_outer_list = list(nodes_new)
nodes_new
x = nodes_new['x']
y = nodes_new['y']
target_nodes_id = ox.nearest_nodes(G, x, y)

# source nodes id
sources_nodes_id = nodes1

# target nodes id
target_nodes_id = target_nodes_id

print("No of elements in list source are:", len(sources_nodes_id))
print("No of elements in list target are:", len(target_nodes_id))

# from object to numeric values to calculate with it (or float)
df['Node ID'] = df['Node ID'].str.strip('[]')
df['Node ID'] = pd.to_numeric(df['Node ID'])
df = df.loc[df['Node ID'].isin(target_nodes_id)]               # Get rows with set of values

df = df.drop(columns='Unnamed: 0')
df = df.drop(columns='Index')
# print(df)   

print("No of elements in list now are:", len(df))

# check
if len(target_nodes_id) == len(df):
    print('Extraction succeeded')
else:
    print('Extraction with error, nodes do not equal target amount of nodes')


df.to_excel("data/final_distances_citycenter_extracted.xlsx")

No of elements in list source are: 6200
No of elements in list target are: 5158
No of elements in list now are: 5145
Extraction with error, nodes do not equal target amount of nodes


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8966f7b1-6d03-4f20-86ea-afcd52732bbc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>